In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from airo_dataset_tools.data_parsers.pose import Pose
np.set_printoptions(precision=3, suppress=True)


In [ ]:
# Find all image with name pose*.png
image_paths = sorted(glob.glob("output/pose_*.png"))
image_paths

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid
images = [cv2.imread(image_path) for image_path in image_paths]

def plot_image_grid(images_bgr, cols=2, scale=4.0):
    images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in images_bgr]
    rows = int(np.ceil(len(images) / cols))

    fig = plt.figure(figsize=(16, 100))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                    nrows_ncols=(rows, cols),
                    axes_pad=0.1,  # pad between axes in inch.
                    )

    # _, axs = plt.subplots(rows, cols, figsize=(10, 60))
    for i, ax in enumerate(grid):
        ax.imshow(images[i])
    plt.show()

plot_image_grid(images)

In [ ]:
from airo_dataset_tools.data_parsers.camera_intrinsics import CameraIntrinsics

# Load camera intrinsics from file
intrinsics_saved = CameraIntrinsics.parse_file("output/camera_intrinsics.json")

def intrinsics_to_K_matrix(intrinsics: CameraIntrinsics):
    fx = intrinsics.focal_lengths_in_pixels.fx
    fy = intrinsics.focal_lengths_in_pixels.fy
    cx = intrinsics.principal_point_in_pixels.cx
    cy = intrinsics.principal_point_in_pixels.cy
    return np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

K = intrinsics_to_K_matrix(intrinsics_saved)
K

In [ ]:
from airo_camera_toolkit.calibration.fiducial_markers import (
    AIRO_DEFAULT_ARUCO_DICT,
    AIRO_DEFAULT_CHARUCO_BOARD,
    detect_aruco_markers,
    get_poses_of_aruco_markers,
    detect_charuco_corners,
    get_pose_of_charuco_board,
    visualize_aruco_detections,
    draw_frame_on_image,
    visualize_charuco_detection,
)

aruco_dict = AIRO_DEFAULT_ARUCO_DICT
charuco_board = AIRO_DEFAULT_CHARUCO_BOARD

def draw_aruco_detections(image, intrinsics):
    # print(intrinsics)
    aruco_result = detect_aruco_markers(image, aruco_dict)

    charuco_result = None
    if aruco_result:
        charuco_result = detect_charuco_corners(image, aruco_result, charuco_board)
        if charuco_result:
            charuco_pose = get_pose_of_charuco_board(charuco_result, charuco_board, intrinsics)

    if aruco_result:
        image = visualize_aruco_detections(image, aruco_result)


    if charuco_result:
        image = visualize_charuco_detection(image, charuco_result)
        if charuco_pose is not None:
            image = draw_frame_on_image(image, charuco_pose, intrinsics)
    return image

In [ ]:
images_with_detections = [draw_aruco_detections(image.copy(), K) for image in images]

In [ ]:
plot_image_grid(images_with_detections)

In [ ]:
from airo_typing import HomogeneousMatrixType


def translational_error(pose1: HomogeneousMatrixType, pose2: HomogeneousMatrixType):
    t1 = pose1[:3, 3]
    t2 = pose2[:3, 3]
    return np.linalg.norm(t1 - t2)

import scipy

def rotational_error(pose1: HomogeneousMatrixType, pose2: HomogeneousMatrixType):
    # phi_6 from: Metrics for 3D Rotations: Comparison and Analysis
    R1 = pose1[:3, :3]
    R2 = pose2[:3, :3]
    return np.linalg.norm(scipy.linalg.logm(R1 @ R2.T))

In [ ]:
from airo_camera_toolkit.calibration.hand_eye_calibration import eye_to_hand_pose_estimation

tcp_poses_in_base = []
marker_poses_in_camera = []

for i, image in enumerate(images):
    image = images[i]
    aruco_result = detect_aruco_markers(image, aruco_dict)
    charuco_result = None
    if aruco_result:
        charuco_result = detect_charuco_corners(image, aruco_result, charuco_board)
        if charuco_result:
            charuco_pose = get_pose_of_charuco_board(charuco_result, charuco_board, K)
            if charuco_pose is not None:
                tcp_pose = Pose.parse_file(f"output/eef_pose_{i:04d}.json").as_homogeneous_matrix()
                tcp_poses_in_base.append(tcp_pose)
                marker_poses_in_camera.append(charuco_pose)

print(len(tcp_poses_in_base))
print(len(marker_poses_in_camera))


calibration_methods = [
    cv2.CALIB_HAND_EYE_TSAI,
    cv2.CALIB_HAND_EYE_PARK,
    cv2.CALIB_HAND_EYE_HORAUD,
    cv2.CALIB_HAND_EYE_ANDREFF,
    cv2.CALIB_HAND_EYE_DANIILIDIS,
]

names = ["Tsai", "Park", "Haraud", "Andreff", "Daniilidis"]

camera_pose = Pose.parse_file("output/camera_extrinsics.json").as_homogeneous_matrix()

for name, method in zip(names, calibration_methods):
    print(f"Method {method}: {name}")
    camera_pose_in_base, error = eye_to_hand_pose_estimation(tcp_poses_in_base, marker_poses_in_camera, method)
    print(camera_pose_in_base)

    error_rot = rotational_error(camera_pose, camera_pose_in_base)
    print(f"System solution {error:.5f}")
    print(f"Rotational error: {error_rot:.5f} rad, {np.rad2deg(error_rot):.3f} deg")
    print(f"Translational error: {translational_error(camera_pose, camera_pose_in_base):.4f}")

# camera_pose_in_base, error = eye_to_hand_pose_estimation(tcp_poses_in_base, marker_poses_in_camera, cv2.CALIB_HAND_EYE_TSAI)
# print(camera_pose_in_base)
# print(error)